## Analyzing Country Profiles With SQL

### Introduction

This project utilizes data from the CIA World Factbook, a compilation of statistics about the countries on earth. The goal of this project is to demonstrate how to analyze country profiles with basic SQL queries.

### Project Overview

<p>The CIA World Factbook contains provides basic intelligence on the history, people, government, economy, energy, geography, environment, communications, transportation, military, terrorism, and transnational issues for 266 world entities. This project will focus on analyzing demographic and country profile information. SQL queries are used to pull the data from the 'Facts' table in the factbook.db database. Below, are descriptions for some of the columns in the table: </p>
<ul>
<li>Name - the name of the country.</li>
<li>Area - the country's total area (both land and water).</li>
<li>area_land - the country's land area in square kilometers.</li>
<li>area_water - the country's water area in square kilometers.</li>
<li>Population - the country's current population.</li>
<li>population_growth - the country's population growth as a percentage.</li>
<li>birth_rate - the number of births per year per 1,000 people.</li>
<li>death_rate - the number of deaths per year per 1,000 people.</li>
</ul>



### Getting Started

In this project, SQLite is used as the database management system (DBMS). The code below is used to connect Jupyter Notebook to the CIA factbook.db database:

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [ ]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Example: To test the connection, this query pulls the first five countries from the facts table:

In [ ]:
%%sql
SELECT *
  FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


<h2>Summary Statistics</h2>
<p>To look for country outliers, summary statistics will need to be performed.</p>


<h3>Country Population & Growth</h3>


<p>The single query written below returns the following:</p>
<ul>
<li>Minimum Population</li>
<li>Maximum Population</li>
<li>Minimum Population Growth</li>
<li>Maximum Population Growth</li>
</ul>
<p>As a result, there's a country with a population of 0 and minimum population growth of 0. The table also includes a country with a population greater than 7256490011 (7.2 billion) and maximum population growth of 4.02.</p>

In [ ]:
%%sql
SELECT MAX(population) AS max_pop, 
       MAX(population_growth) AS max_growth, 
       MIN(population) AS min_pop, 
       MIN(population_growth) AS min_pop_growth
  FROM facts;

Done.


max_pop,max_growth,min_pop,min_pop_growth
7256490011,4.02,0,0.0


<p>Subqueries are used below to further investivate these two countries without using specific values. The query below returns the country with the minimum population, which is Antarctica. The data is consistent with the CIA Factbook as Antarctica <i>"has no indigenous inhabitants, but there are both permanent and summer-only staffed research stations."</i></p>

In [ ]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MIN(population)
                              FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


<p>Next, the query below returns the country with the maximum population. It appears that the table contains a row for the whole world, which explains the population of over 7.2 billion.</p>

In [ ]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MAX(population)
                              FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


<p>Now that we know there's a row for the whole world, it's best practice to reccalculate the population summary statistics. The query below excludes the row for the whole world:</p>

In [ ]:
%%sql
SELECT MAX(population) AS max_pop, 
       MAX(population_growth) AS max_growth, 
       MIN(population) AS min_pop, 
       MIN(population_growth) AS min_pop_growth
  FROM facts
 WHERE name <> 'World'

Done.


max_pop,max_growth,min_pop,min_pop_growth
1367485388,4.02,0,0.0


### Average Population &amp; Area

<p>The query below calculates the average value for the population and area columns in the table, excluding the world row. The average population is around 32 million and the average area is 555 thousand square kilometers. </p>

In [ ]:
%%sql
SELECT ROUND(AVG(population), 2) AS avg_pop,
       ROUND(AVG(area), 2) AS avg_area
  FROM facts
 WHERE name <> 'World'

Done.


avg_pop,avg_area
32242666.57,555093.55


### Density

<p>Density depends on the population and the country's area. In the U.S., population density is expressed as the number of people per square mile of land area. The query below returns all countries that are densely populated, as well as meeting both of the following criteria:</p>
<ul>
<li>The population is above average.</li>
<li>The area is below average.</li>
</ul>
<p>Therefore, the countries in this query are densly populated given their high population and low area.</p>

In [ ]:
%%sql
SELECT name, population, area
  FROM facts
 WHERE population>(SELECT AVG(population)
                    FROM facts
                   WHERE name <> 'World') AND area<(SELECT AVG(area)
                                                    FROM facts
                                                    WHERE name <> 'World')

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


<p><strong>Population-to-Area Ratio:</strong> The previous query calculated countries that had higher-than-average populations with lower-than-average area. The formula for calculating population density is Dp= N/A. In this equation, Dp is the density of population, N is the total population as a number of people, and A is the land area covered by that population. Based on this equation, Bangladesh has a population-to-area ratio of 1,138 (168,957,745/148,460). If we remove the population and area criteria, we can see that there are 11 more countries with a much higher population-to-area ratio, with Macau being on top with a ratio of 21,168. The top three countries in this list (Macau, Monaco, and Singapore) are all incredibly small countries with dense populations, thus contributing to their high ratios.</p>

In [ ]:
%%sql
SELECT name, population, area, ROUND(CAST(population AS float)/CAST(area AS float), 2) AS pop_area_ratio
  FROM facts
 WHERE name <> 'World'
ORDER BY pop_area_ratio DESC
LIMIT 10;

Done.


name,population,area,pop_area_ratio
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


<h3>Population Growth Next Year</h3>
<p>Next, the query below returns the top 10 countries that will add the most people to their populations next year, excluding the world row. Therefore, India will add the most number of people to its population in the next year. To note, India does not have the highest population_growth amongst the top 10 countries. However, this growth is due to the country's large current population that puts India with the highest possibility to add the most number of people in the next year by about ~15.2 million people! That's more than twice the amount of the next highest country, China with an estimated population growth of ~6.2 million.</p>

In [ ]:
%%sql
SELECT  name, population AS current_pop, population_growth, 
        ROUND(population*population_growth, 2) AS Population_growth_next_year 
  FROM facts
 WHERE name <> 'World'
ORDER BY Population_growth_next_year DESC
LIMIT 10;

Done.


name,current_pop,population_growth,Population_growth_next_year
India,1251695584,1.22,1527068612.48
China,1367485388,0.45,615368424.6
Nigeria,181562056,2.45,444827037.2
Pakistan,199085847,1.46,290665336.62
Ethiopia,99465819,2.89,287456216.91
Bangladesh,168957745,1.6,270332392.0
United States,321368864,0.78,250667713.92
Indonesia,255993674,0.92,235514180.08
"Congo, Democratic Republic of the",79375136,2.45,194469083.2
Philippines,100998376,1.61,162607385.36


<h3>Water to Land Ratio</h3>
<p>The query below returns those countries with the highest water to land ratios, excluding the world row. British Indian Ocean Territory has a high proportion of water in comparison to it's land. This territory is made of an archipelago of 58 islands covering some 640,000 sq km of ocean. Furthermore, the British Indian Ocean Territory and Virgin Islands are the only countries to have the area of water greater than land.</p>

In [ ]:
%%sql
SELECT name, area_water, area_land,
       ROUND(CAST(area_water AS Float)/CAST(area_land AS Float), 2) AS Water_land_ratio
  FROM facts
 WHERE name <> 'World'
ORDER BY Water_land_ratio DESC
 LIMIT 10;

Done.


name,area_water,area_land,Water_land_ratio
British Indian Ocean Territory,54340,60,905.67
Virgin Islands,1564,346,4.52
Puerto Rico,4921,8870,0.55
"Bahamas, The",3870,10010,0.39
Guinea-Bissau,8005,28120,0.28
Malawi,24404,94080,0.26
Netherlands,7650,33893,0.23
Uganda,43938,197100,0.22
Eritrea,16600,101000,0.16
Liberia,15049,96320,0.16


<h3>Birth & Death Rates</h3>
<p>The query below returns the top 10 countries that have a higher death rate than birth rate, in descending order. This is evident with the decline in population. </p>

In [ ]:
%%sql
SELECT name, death_rate, birth_rate
  FROM facts
 WHERE death_rate>birth_rate
ORDER BY death_rate DESC
 LIMIT 10;

Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6
Serbia,13.66,9.08
Belarus,13.36,10.7
Hungary,12.73,9.16
Moldova,12.59,12.0
Estonia,12.4,10.51


<p>Next, the query below returns the country that has the most people, which is China at ~1.37 billion people.</p>

In [ ]:
%%sql
SELECT *
    FROM facts
    WHERE population = (SELECT MAX(population)
                           FROM facts
                           WHERE id != 261)
    AND id != 261;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


<p>This query returns the country that has the highest growth rate. The results shows that South Sudan has the highest population growth at a rate of 4.02%. This is evident because the country's birth rate (36.91%) largely surpasses the country's death rate (8.18%)</p>

In [ ]:
%%sql
SELECT *
    FROM facts
    WHERE population_growth = (SELECT MAX(population_growth)
                           FROM facts
                           WHERE id != 261)
    AND id != 261;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


<p>Which country has the highest birth Rate? Well, we can see that Niger is the country with the highest birth rate. Also, the top 20 countries with the highest birth rates are all African countries.</p>

In [ ]:
%%sql
SELECT name, birth_rate, population_growth
    FROM facts
    WHERE id != 261
    ORDER BY birth_rate DESC
    LIMIT 20;

Done.


name,birth_rate,population_growth
Niger,45.45,3.25
Mali,44.99,2.98
Uganda,43.79,3.24
Zambia,42.13,2.88
Burkina Faso,42.03,3.03
Burundi,42.01,3.28
Malawi,41.56,3.32
Somalia,40.45,1.83
Angola,38.78,2.78
Mozambique,38.58,2.45


<p>In descending order, we can see that Ukraine has the highest death rate in the world, at 14.46% and a death/birth rate ratio of 1.35%. Additionally, Bulgaria has the second highest death rate at 14.44% and a death/birth rate ratio of 1.62%.</p>

In [ ]:
%%sql
SELECT name, death_rate, birth_rate, ROUND(death_rate / birth_rate, 2) as death_birth_ratio
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY death_rate DESC
    LIMIT 20;

Done.


name,death_rate,birth_rate,death_birth_ratio
Ukraine,14.46,10.72,1.35
Bulgaria,14.44,8.92,1.62
Latvia,14.31,10.0,1.43
Lithuania,14.27,10.1,1.41
Russia,13.69,11.6,1.18
Serbia,13.66,9.08,1.5
Belarus,13.36,10.7,1.25
Hungary,12.73,9.16,1.39
Moldova,12.59,12.0,1.05
Estonia,12.4,10.51,1.18


<p>Lastly, the query below returns the Population-to-Area Ratio for the top 20 countries in descending order. A previous query was ran for the top 10 countries, excluding the world row.</p>

In [ ]:
%%sql
SELECT name, population, area, ROUND(population/area, 2) as pop_area_ratio
    FROM facts
    ORDER BY pop_area_ratio DESC
    LIMIT 20;

Done.


name,population,area,pop_area_ratio
Macau,592731,28,21168.0
Monaco,30535,2,15267.0
Singapore,5674472,697,8141.0
Hong Kong,7141106,1108,6445.0
Gaza Strip,1869055,360,5191.0
Gibraltar,29258,6,4876.0
Bahrain,1346613,760,1771.0
Maldives,393253,298,1319.0
Malta,413965,316,1310.0
Bermuda,70196,54,1299.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=376dca8b-085a-40ab-8346-99bd0448301c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>